In [ ]:
# %pip install langchain
# %pip install langchain-openai
# %pip install langchainhub
# %pip install pypdf
# %pip install chromadb

In [4]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
import os

def _set_env_from_file(var: str, file_path: str = "openai_key.txt"):
    """
    Reads an API key from a specified file and sets it as an environment variable.
    """
    if not os.environ.get(var):
        try:
            # The 'with open' statement ensures the file is closed automatically
            with open(file_path, 'r') as f:
                # Read the first line and strip any leading/trailing whitespace
                key = f.readline().strip()

            if key:
                os.environ[var] = key
                print(f"Successfully loaded {var} from {file_path}")
            else:
                print(f"Warning: {file_path} is empty.")

        except FileNotFoundError:
            print(f"Error: Key file not found at {file_path}. Please create the file.")

# --- Execution ---
# Set the environment variable OPENAI_API_KEY from the file
_set_env_from_file('OPENAI_API_KEY')

# Verify it was set (optional)
# print(os.environ.get('OPENAI_API_KEY'))

In [3]:
MODEL="llama3.2"

In [5]:
pdf_path="C:\\Users\\gyanr\\gyan-python-workspace\\DSC630\\why-kmeans-research-paper.pdf"
loader=PyPDFLoader(pdf_path)
pdf_docs = loader.load_and_split()
pdf_docs

[Document(metadata={'producer': 'GPL Ghostscript 8.54', 'creator': 'PDFCreator Version 0.9.3', 'creationdate': '2009-08-05T10:21:00-04:00', 'moddate': 'D:20090805102100', 'title': 'Jain_PRL_paper_Aug4', 'author': 'Abhishek', 'keywords': '', 'subject': '', 'source': 'C:\\Users\\gyanr\\gyan-python-workspace\\DSC630\\why-kmeans-research-paper.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='To appear in Pattern Recognition Letters, 2009.  \nData Clustering: 50 Years Beyond K-Means1 \n \nAnil K. Jain \nDepartment of Computer Science & Engineering \nMichigan State University \nEast Lansing, Michigan 48824 USA \njain@cse.msu.edu  \n \n \nAbstract:  Organizing data into sensible groupings is one of the most fundamental \nmodes of understanding and learning. As an example, a common scheme of scientific \nclassification puts organisms into a system of ranked taxa: domain, kingdom, phylum, \nclass, etc.. Cluster analysis is the formal study of methods and algorithms for grou

In [6]:
doc_obj = pdf_docs[0]
doc_obj

Document(metadata={'producer': 'GPL Ghostscript 8.54', 'creator': 'PDFCreator Version 0.9.3', 'creationdate': '2009-08-05T10:21:00-04:00', 'moddate': 'D:20090805102100', 'title': 'Jain_PRL_paper_Aug4', 'author': 'Abhishek', 'keywords': '', 'subject': '', 'source': 'C:\\Users\\gyanr\\gyan-python-workspace\\DSC630\\why-kmeans-research-paper.pdf', 'total_pages': 35, 'page': 0, 'page_label': '1'}, page_content='To appear in Pattern Recognition Letters, 2009.  \nData Clustering: 50 Years Beyond K-Means1 \n \nAnil K. Jain \nDepartment of Computer Science & Engineering \nMichigan State University \nEast Lansing, Michigan 48824 USA \njain@cse.msu.edu  \n \n \nAbstract:  Organizing data into sensible groupings is one of the most fundamental \nmodes of understanding and learning. As an example, a common scheme of scientific \nclassification puts organisms into a system of ranked taxa: domain, kingdom, phylum, \nclass, etc.. Cluster analysis is the formal study of methods and algorithms for group

In [7]:
from IPython.display import display, Markdown

Markdown(doc_obj.page_content)

To appear in Pattern Recognition Letters, 2009.  
Data Clustering: 50 Years Beyond K-Means1 
 
Anil K. Jain 
Department of Computer Science & Engineering 
Michigan State University 
East Lansing, Michigan 48824 USA 
jain@cse.msu.edu  
 
 
Abstract:  Organizing data into sensible groupings is one of the most fundamental 
modes of understanding and learning. As an example, a common scheme of scientific 
classification puts organisms into a system of ranked taxa: domain, kingdom, phylum, 
class, etc.. Cluster analysis is the formal study of methods and algorithms for grouping, or 
clustering, objects according to measured or perceived intrinsic characteristics or 
similarity. Cluster analysis does not use category labels that tag objects with prior 
identifiers, i.e., class labels. The absence of category information distinguishes data 
clustering (unsupervised learning) from classification or discriminant analysis 
(supervised learning). The aim of clustering is to find structure in data and is therefore 
exploratory in nature. Clustering has a long and rich history in a variety of scientific 
fields. One of the most popular and simple clustering algorithms, K-means, was first 
published in 1955. In spite of the fact that K-means was proposed over 50 years ago and 
thousands of clustering algorithms have been published since then, K-means is still  
widely used. This speaks to the difficulty of designing a general purpose clustering 
algorithm and the ill-posed problem of clustering. We provide a brief overview of 
clustering, summarize well known clustering methods, discuss the major challenges and 
key issues in designing clustering algorithms, and point out some of the emerging and 
useful research directions, including semi-supervised clustering, ensemble clustering, 
simultaneous feature selection during data clustering and large scale data clustering. 
 
 
1.  Introduction 
 
Advances in sensing and storage technology and dramatic growth in applications such as 
Internet search, digital imaging, and video surveillance have created many high-volume, 
high-dimensional data sets. It is estimated that the digital universe consumed 
approximately 281 exabytes in 2007, and it is projected to be 10 times that size by 2011. 
(One exabyte is ~10 
18 bytes or 1,000,000 terabytes) [Gantz, 2008]. Most of the data is 
stored digitally in electronic media, thus providing huge potential for the development of 
automatic data analysis, classification, and retrieval techniques. In addition to the growth 
in the amount of data, the variety of available data (text, image, and video) has also 
increased. Inexpensive digital and video cameras have made available huge archives of 
                                                 
1 This paper is based on the King-Sun Fu Prize lecture delivered at the 19 th  International 
Conference on Pattern Recognition (ICPR), Tampa, FL, December 8, 2008.

In [8]:
embeddings = OpenAIEmbeddings()
embeddings

vectordb = Chroma.from_documents(pdf_docs,embedding=embeddings)

retriever = vectordb.as_retriever()

llm = ChatOpenAI(model=MODEL,temperature=0 )

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
#prompt

question_ans_chain = 